In [ ]:
import findspark
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import io
import numpy as np
import pandas as pd
from pyspark.sql.functions import avg
from math import sqrt

# Initialize Spark
findspark.init()

# Build Spark session
print("Building Spark session...")
try:
    spark = SparkSession.builder \
        .getOrCreate()
except Exception as e:
    print("Exception while creating Spark session:")
    print(e)

# Set up data stream from specified host and port
print("Setting up stream...")
host = 'localhost'
port = 4545
lines = spark.readStream \
    .format("socket") \
    .option("host", host) \
    .option("port", port) \
    .load()

# Prepare data schema, including sensor IDs
print("Preparing data schema...")
sensor_ids = ["CAT17", "CB1101", "CB1142", "CB1143", "CB1599", "CB1699", "CB2105", "CB02411", 
                  "CEE016", "CEK18", "CEK31", "CEK049", "CEV011", "CJE181", "CJM90", "CLW239", 
                  "COM205", "CVT387"]  # list of sensor ids
schema_string = "Timestamp STRING, Time_gap INT, " + ", ".join([f"Count_{sensor_id} INT, Average_speed_{sensor_id} INT" for sensor_id in sensor_ids])

# Transform incoming data to match schema
print("Transforming data...")
csv_data = lines \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_csv("value", schema_string).alias("data")) \
    .select("data.*")

# Function to process each batch of data
def process_batch(df, epoch_id):
    print("Checking for empty dataframe...")
    # If dataframe is empty, exit function
    if df.rdd.isEmpty():
        return
    
    print("Print Dataframe shema")
    df.printSchema()
    print("Print Dataframe columns")
    print(df.columns)

    print("Processing batch...")

    # Convert columns to appropriate types
    print("Converting columns...")
    df = df.withColumn("Time_gap", df["Time_gap"].cast(IntegerType()))

    # Lists to hold correlation data
    correlations_count = []
    correlations_speed = []

    # Loop over pairs of sensors
    for i in range(len(sensor_ids)):
        for j in range(i + 1, len(sensor_ids)):
            sensor_id1 = sensor_ids[i]
            sensor_id2 = sensor_ids[j]

            count_column1 = f"Count_{sensor_id1}"
            speed_column1 = f"Average_speed_{sensor_id1}"
            count_column2 = f"Count_{sensor_id2}"
            speed_column2 = f"Average_speed_{sensor_id2}"

            # Convert sensor columns to integer
            df = df.withColumn(count_column1, df[count_column1].cast(IntegerType()))
            df = df.withColumn(speed_column1, df[speed_column1].cast(IntegerType()))
            df = df.withColumn(count_column2, df[count_column2].cast(IntegerType()))
            df = df.withColumn(speed_column2, df[speed_column2].cast(IntegerType()))

            # Compute means for correlation calculation
            count_mean1 = df.agg(avg(count_column1)).first()[0]
            speed_mean1 = df.agg(avg(speed_column1)).first()[0]
            count_mean2 = df.agg(avg(count_column2)).first()[0]
            speed_mean2 = df.agg(avg(speed_column2)).first()[0]

            # Compute Pearson correlation between sensor pairs
            print(f"Calculating Pearson correlation for sensors {sensor_id1} and {sensor_id2}...")
            # The following lines calculate the numerator and denominator of the Pearson correlation coefficient
            df = df.withColumn("count_product", (df[count_column1] - count_mean1) * (df[count_column2] - count_mean2))
            df = df.withColumn("speed_product", (df[speed_column1] - speed_mean1) * (df[speed_column2] - speed_mean2))
            df = df.withColumn("count_square1", (df[count_column1] - count_mean1) ** 2)
            df = df.withColumn("speed_square1", (df[speed_column1] - speed_mean1) ** 2)
            df = df.withColumn("count_square2", (df[count_column2] - count_mean2) ** 2)
            df = df.withColumn("speed_square2", (df[speed_column2] - speed_mean2) ** 2)

            count_product_sum = df.agg({"count_product": "sum"}).first()[0]
            speed_product_sum = df.agg({"speed_product": "sum"}).first()[0]
            count_square_sum1 = df.agg({"count_square1": "sum"}).first()[0]
            speed_square_sum1 = df.agg({"speed_square1": "sum"}).first()[0]
            count_square_sum2 = df.agg({"count_square2": "sum"}).first()[0]
            speed_square_sum2 = df.agg({"speed_square2": "sum"}).first()[0]

            # Compute the correlation coefficient, protecting against division by zero
            if (sqrt(count_square_sum1) * sqrt(count_square_sum2)) != 0:
                corr_count = count_product_sum / (sqrt(count_square_sum1) * sqrt(count_square_sum2))
            else:
                corr_count = None

            if (sqrt(speed_square_sum1) * sqrt(speed_square_sum2)) != 0:
                corr_speed = speed_product_sum / (sqrt(speed_square_sum1) * sqrt(speed_square_sum2))
            else:
                corr_speed = None

            # Print correlation results
            print(f"Pearson correlation coefficient between {count_column1} and {count_column2}: {corr_count}")
            print(f"Pearson correlation coefficient between {speed_column1} and {speed_column2}: {corr_speed}")
            # Add correlation results to lists
            correlations_count.append(((sensor_id1, sensor_id2), corr_count))
            correlations_speed.append(((sensor_id1, sensor_id2), corr_speed))

    # Sort correlation lists and print top 5 for each sensor type
    correlations_count = [x for x in correlations_count if x[1] is not None]
    correlations_speed = [x for x in correlations_speed if x[1] is not None]

    correlations_count.sort(key=lambda x: x[1], reverse=True)
    correlations_speed.sort(key=lambda x: x[1], reverse=True)

    print("Top 5 correlations for Count:")
    for i in range(__builtins__.min([5, len(correlations_count)])):
        print(f"{correlations_count[i][0][0]} and {correlations_count[i][0][1]}: {correlations_count[i][1]}")

    print("Top 5 correlations for Average speed:")
    for i in range(__builtins__.min([5, len(correlations_speed)])):
        print(f"{correlations_speed[i][0][0]} and {correlations_speed[i][0][1]}: {correlations_speed[i][1]}")

# Set up stream query with batch processing function and processing time trigger
print("Setting up stream query...")
output_query = csv_data.writeStream \
    .foreachBatch(process_batch) \
    .trigger(processingTime="5 seconds") \
    .outputMode("update") \
    .start()

# Await termination of stream query
print("Awaiting termination...")
output_query.awaitTermination()

Creating Spark configuration...
Building Spark session...


23/05/21 20:43:02 WARN Utils: Your hostname, vladi-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/05/21 20:43:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/21 20:43:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/21 20:43:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Setting up stream...


23/05/21 20:43:05 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


Preparing data schema...
Transforming data...


...

Pearson correlation coefficient between Count_CB1143 and Count_CEK18: 0.8268656946644957
Calculating Pearson correlation for sensors CB1143 and CEK31...
Pearson correlation coefficient between Count_CB1143 and Count_CEK31: 0.7472043570432472
Calculating Pearson correlation for sensors CB1143 and CEK049...
Pearson correlation coefficient between Count_CB1143 and Count_CEK049: 0.6958360596997855
Calculating Pearson correlation for sensors CB1143 and CEV011...
Pearson correlation coefficient between Count_CB1143 and Count_CEV011: 0.8442829358426123
Calculating Pearson correlation for sensors CB1143 and CJE181...
Pearson correlation coefficient between Count_CB1143 and Count_CJE181: 0.8382309111720204
Calculating Pearson correlation for sensors CB1143 and CJM90...
Pearson correlation coefficient between Count_CB1143 and Count_CJM90: 0.7151985836646365
Calculating Pearson correlation for sensors CB1143 and CLW239...
Pearson correlation coefficient between Count_CB1143 and Count_CLW239: 0.62

23/05/21 21:36:09 WARN TextSocketMicroBatchStream: Stream closed by localhost:4545


Calculating Pearson correlation for sensors CB02411 and CEK18...
Pearson correlation coefficient between Count_CB02411 and Count_CEK18: 0.872618065562197
Calculating Pearson correlation for sensors CB02411 and CEK31...
Pearson correlation coefficient between Count_CB02411 and Count_CEK31: 0.7522978557301021
Calculating Pearson correlation for sensors CB02411 and CEK049...
Pearson correlation coefficient between Count_CB02411 and Count_CEK049: 0.7493205225623696
Calculating Pearson correlation for sensors CB02411 and CEV011...
Pearson correlation coefficient between Count_CB02411 and Count_CEV011: 0.8731460375472878
Calculating Pearson correlation for sensors CB02411 and CJE181...
Pearson correlation coefficient between Count_CB02411 and Count_CJE181: 0.854723910373691
Calculating Pearson correlation for sensors CB02411 and CJM90...
Pearson correlation coefficient between Count_CB02411 and Count_CJM90: 0.7512987496347532
Calculating Pearson correlation for sensors CB02411 and CLW239...


Pearson correlation coefficient between Count_COM205 and Count_CVT387: 0.8212533413955088
Top 5 correlations for Count:
CB1599 and CVT387: 0.926881526058239
CB1599 and CB1699: 0.925703869816148
CAT17 and CEK18: 0.9210610930943841
CB1699 and CB02411: 0.9001807291414262
CB1699 and CVT387: 0.8997708366967835


23/05/21 21:36:55 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 5000 milliseconds, but spent 100532 milliseconds
